In [ ]:
# Setup and Libraries
#This cell installs the necessary optimization libraries for Quantization and Fine-tuning*/
# Install optimization libraries
!pip install -q auto-gptq optimum peft datasets accelerate bitsandbytes

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Data Preprocessing (The "X")
# This logic cleans the raw symptom data, removing underscores and grouping them by disease.
# Load dataset
dis_symp_df = pd.read_csv("/kaggle/input/disease-symptom-prediction/dataset.csv")

# Clean symptoms and group by disease
symptom_cols = [col for col in dis_symp_df.columns if col.startswith("Symptom")]
dis_symp_df[symptom_cols] = dis_symp_df[symptom_cols].replace("_", " ", regex=True)

# Combine symptoms into a single string for RAG
dis_symp_df["Symptoms"] = dis_symp_df[symptom_cols].apply(lambda row: ', '.join([s for s in row if pd.notnull(s)]), axis=1)
dis_symp_df_grouped = dis_symp_df.groupby("Disease")["Symptoms"].apply(lambda x: ", ".join(set(x))).reset_index()

In [ ]:
# The RAG & Quantization Logic (The "Z" and "Y")
# This part implements the 4-bit Quantization and Semantic Search you mentioned in your achievements.
# Initialize Embeddings for Semantic Search
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
corpus = dis_symp_df_grouped.apply(lambda row: f"Disease: {row['Disease']}. Symptoms: {row['Symptoms']}", axis=1).tolist()
corpus_embeddings = embed_model.encode(corpus, convert_to_tensor=True)

# Configure 4-bit Quantization (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load Llama-2 Model
model_name = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
# Fine-Tuning Setup (The "Z")
# This implements the PEFT/LoRA parameters used to adapt the model to medical data.
# Prepare model for PEFT/LoRA training
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8, 
    lora_alpha=16, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)